Each leaf bloq reports its "leaf costs". The costs are percolated up the DAG with simple recursion:

```python
def cost(caller):
    # Base case: leaf bloq
    if is_leaf(caller):
        return caller.my_leaf_costs()
    
    # Recursive step: callees
    return sum(n * cost(callee) for n, callee in caller.callees())
```

Note that we *only* query `bloq.leaf_costs()` when the bloq is an actual leaf. What if there is other information we want to annotate on bloqs that are not leaf bloqs? Like succeess probability or some sort of overhead? We add a method for including costs that should be added to the ones gleaned from recursion.

```python
    cost = sum(n * cost(callee) for n, callee in caller.callees())
    cost += caller.my_static_costs()
    return cost
```

I've been using addition and multiplication for convenience, but these operations depend on the cost value type. `+` should be considered a generic reduction step and `* n` should mean "do that reduction `n` times". For something like success probability, you'd use multiplication and power. For something like max qubit width, you'd use max and identity. Each type of cost also has to report its "identity" aka 0 value to make the reduction work.

I've been using one cost value; but the `.my_leaf_costs()` and `.my_static_costs()` methods will return a list of `(costkey, costval)` tuples and all the operations should be done be joining on the `costkey` column. 

-----

We want the source code to define a canonical call graph but we may want to muck with it for resource estimation purposes. For example: we already support stopping early by marking certain bloqs as leaf bloqs; and we support merging bloqs through the `generalizer`. 

Consider if we wanted to query either Total T Count or T+Tof count. With one "costs" method; you'd either never get the total T count or always get the total T count. If you did something really naive, you'd double-count the Ts from the toffoli decomposition. We need to be able to say that it costs *either* the sum of its callees *or* this fixed value of 1 toffoli.

So in our design, we first build the call graph in the usual way and generalize and leaf-ify to our hearts content. We then use that online call graph to do the cost recursion which will intelligently dispatch to either the recursive costs or the leaf costs.

A potential drawback is if you leafify something that doesn't explicitly specify its leaf costs, you may quietly miss some important costs. But it would be redundant to manually put in all the recursive costs in case that bloq is ever used as a leaf. The compromise solution is for the default leaf costs method to report that it takes 1 bloq-count of `self`. This prevents accidently ignoring a whole subtree of the call graph. But any of the non-bloq-count recursively-derived costs would be lost.

In [ ]:
from qualtran.drawing import show_bloq, show_call_graph

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from qualtran.bloqs.for_testing.costing import CostingBloq, make_example_1
cost_bloq = make_example_1()
g, _ = cost_bloq.call_graph(keep=lambda b:b.name in ['TGate'])

In [ ]:
show_call_graph(g)

In [ ]:
for bloq, data in g.nodes.data():
    print(bloq,'\t', data['costs'])

In [ ]:
from qualtran.resource_counting.bloq_counts import _sum_up_costs
totals = _sum_up_costs(g)
print('----------------\n\n')

for bloq, costs in totals.items():
    print(bloq)
    for costkey, costval in costs.items():
        print('  ', costkey, '\t', costval)
    print()

In [ ]:
from qualtran.drawing.bloq_counts_graph import GraphvizCounts

class CostGraphviz(GraphvizCounts):
    def __init__(self, g, totals):
        super().__init__(g)
        self.totals = totals
    
    
    def get_node_properties(self, b):
        label = ['<']
        label += ['<font point-size="10">']
        label += ['<table border="0" cellborder="1" cellspacing="0" cellpadding="5">\n']
        label += [f'<tr><td colspan="2"><font point-size="12">{b.name}(nq={b.num_qubits})</font></td></tr>\n']
        
        for cost, costval in self.totals[b].items():
            label += [f'<tr><td>{cost}</td><td>{costval}</td></tr>']
        
        label += ['</table></font>']
        label += ['>']
        return {'label': ''.join(label), 'shape': 'plaintext'}
    
    
CostGraphviz(g, totals).get_svg()